# NanoGPT on Colab/Kaggle (GPU)

Use this notebook to train the full 10.8M-parameter NanoGPT on a free GPU (Colab/Kaggle). Sections include environment setup, Drive mounting, installs, file uploads, GPU enablement, collaboration, and exporting results.

## 1. Setup and Authentication
- Verify runtime (Colab/Kaggle) and Python version
- (Optional) Authenticate Google account if accessing private Drive files
- Set working directory

In [ ]:
import sys, platform, os, subprocess
print(f"Python: {sys.version.split()[0]}")
print(f"Platform: {platform.platform()}")
print(f"Working dir: {os.getcwd()}")
# Uncomment if you need Google auth for private files (Colab only)
# from google.colab import auth
# auth.authenticate_user()


## 2. Mount Google Drive
- (Colab) Mount Drive to read/write datasets and checkpoints
- (Kaggle) Skip this; use /kaggle/working instead

In [ ]:
# Colab only: mount Drive (skip on Kaggle)
# from google.colab import drive
# drive.mount('/content/drive')

# Set working directory (Colab default is /content)
# os.chdir('/content')
# print('CWD:', os.getcwd())

## 3. Install Required Libraries
- Ensure PyTorch is present (Colab/Kaggle usually has it)
- Install/upgrade as needed

In [ ]:
# Check torch; install if missing or to upgrade
import importlib, subprocess, sys
try:
    importlib.import_module('torch')
    import torch
    print('Torch version:', torch.__version__)
except ImportError:
    print('Installing torch...')
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'torch', 'torchaudio', 'torchvision', '--index-url', 'https://download.pytorch.org/whl/cu121'])

## 4. Upload and Access Files
- Use Colab file upload widget or Kaggle file upload
- Download Tiny Shakespeare automatically if not present

In [ ]:
import os, urllib.request
DATA_URL = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
DATA_PATH = "input.txt"
if not os.path.exists(DATA_PATH):
    print("Downloading input.txt ...")
    urllib.request.urlretrieve(DATA_URL, DATA_PATH)
    print("Done.")
else:
    print("Found input.txt")

## 5. Use GPU/TPU Resources
- Ensure GPU is selected (Runtime > Change runtime type > GPU)
- Verify CUDA availability and device
- Define and train the 10.8M-parameter NanoGPT
- Monitor progress with periodic logging

In [ ]:
import torch, torch.nn as nn
from torch.nn import functional as F

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Device:', device)

# Hyperparameters for 10.8M param model
batch_size = 64
block_size = 256
max_iters = 6000
eval_interval = 500
learning_rate = 3e-4
eval_iters = 200
n_embd = 384
n_head = 6
n_layer = 6
dropout = 0.2

with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(list(set(text)))
vocab_size = len(chars)
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data_split = train_data if split == 'train' else val_data
    ix = torch.randint(len(data_split) - block_size, (batch_size,))
    x = torch.stack([data_split[i:i + block_size] for i in ix])
    y = torch.stack([data_split[i + 1:i + block_size + 1] for i in ix])
    return x.to(device), y.to(device)

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)
        wei = q @ k.transpose(-2, -1) * k.shape[-1] ** -0.5
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        v = self.value(x)
        out = wei @ v
        return out

class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)
    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )
    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)
    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class GPTLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)
        self.apply(self._init_weights)
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
    def forward(self, idx, targets=None):
        B, T = idx.shape
        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)
        loss = None
        if targets is not None:
            logits = logits.view(B*T, -1)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, _ = self(idx_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

model = GPTLanguageModel().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

print(f"Parameters: {sum(p.numel() for p in model.parameters())/1e6:.2f}M")


In [ ]:
import time, sys
start_time = time.time()
last_print = start_time

for iter in range(max_iters):
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        elapsed = time.time() - start_time
        steps_per_sec = iter / elapsed if iter > 0 else 0
        eta = (max_iters - iter) / steps_per_sec / 60 if steps_per_sec > 0 else 0
        print(f"Step {iter:5d}/{max_iters} | Train {losses['train']:.4f} | Val {losses['val']:.4f} | {elapsed/60:.1f}m | ETA {eta:.1f}m")
        sys.stdout.flush()
    elif iter > 0 and iter % 100 == 0:
        now = time.time()
        if now - last_print > 5:
            elapsed = now - start_time
            sps = iter / elapsed
            eta = (max_iters - iter) / sps / 60
            print(f"Step {iter:5d}/{max_iters} | {sps:.2f} it/s | ETA {eta:.0f}m", end='\r')
            sys.stdout.flush()
            last_print = now

    xb, yb = get_batch('train')
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print("\nTraining done. Generating sample...\n")
context = torch.zeros((1, 1), dtype=torch.long, device=device)
gen = model.generate(context, max_new_tokens=500)
print(decode(gen[0].tolist()))

## 6. Share and Collaborate
- Use Colab sharing (top-right Share) or Kaggle notebook sharing
- Manage permissions for view/edit

## 7. Export Results
- Save checkpoints and generated samples to Drive or local files
- Download artifacts for submission

In [ ]:
# Save a checkpoint and a generated sample
ckpt_path = 'nanogpt_checkpoint.pt'
sample_path = 'sample.txt'

torch.save({'model': model.state_dict()}, ckpt_path)
with open(sample_path, 'w', encoding='utf-8') as f:
    f.write(decode(gen[0].tolist()))

print('Saved:', ckpt_path, sample_path)

# If on Colab Drive, these will be in /content/drive/... after mounting
# If on Kaggle, they will be in /kaggle/working and downloadable from the sidebar